In [1]:
import spacy
from spacy.tokens import Doc
class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)



### Read from File 
**This Function Reads the file and returns the content of files in list format**


In [2]:
import os
def read_file(filename):
    file = open('/home/rk/Downloads/HCL ML Challenge/HCL ML Challenge Dataset/'+filename, 'r')
    list_file_content = file.readlines()
    #print(list_file_content)
    return list_file_content

### Filtering of Data
**This Function takes string input. It then removes extra spacing and also '\n' character if present**

In [3]:
def clean_lis(ele):
    ele=ele.strip()
    ele=" ".join(ele.split())
    if ele=="\n":
        ele=""
    return ele

### Categorizing type of file
**After Cleaning the list of file contents, the file is categorised into 5 categories : <br>
Category 0: It is a Balance sheet for 2 years but but none of them is 2019             (eg: '2013 2014')<br>
Category 1: It is a Balance sheet for 2 years and first year is 2019 in Balance Sheet  (eg: '2019 2020')<br>
Category 2: It is a Balance sheet for 2 years and second year is 2019 in Balance Sheet (eg: '2018 2019')<br>
Category 3: It is a Balance sheet for the year 2019 only                               (eg: '2018 2019')<br>
Category 4: It is a Balance sheet for 1 year and that year is not 2019                 (eg: '2018 2019')<br>
This function returns the Category along with index from where the category is extracted**

In [4]:
def check_2019(filtered_list):
    idx=0
    if filtered_list[0]=="Balance Sheet" and "As at"  in filtered_list[1]:
            x=filtered_list[1].split()
            if x[-1]=="2019":
                return 3,3
    for i in filtered_list:
        x=i.split()
        if (len(x)==1 and x[0].isnumeric()): 
            if(x[0]=="2019"):
                return 3,idx
            else:
                return 4,idx
                
        if len(x)>1 and len(x[-1])>3 and len(x[-2])>3 and "Notes" in x and x[-1][-4:].isnumeric() and x[-2][-4:].isnumeric():
            if x[-1][-4:]=="2019":
                return 2,idx
            if x[-2][-4:]=="2019":
                return 1,idx
            else:
                return 0,idx
        
        if "Notes" in x and len(x[-1])>3 and x[-1][-4:].isnumeric():
             if x[-1][-4:]=="2019":
                return 3,idx
             else:
                return 4,idx
                
        if len(x)>1 and len(x[-1])>3 and len(x[-2])>3 and x[-1][-4:].isnumeric() and x[-2][-4:].isnumeric():
            if x[-1][-4:]=="2019":
                return 2,idx
            if x[-2][-4:]=="2019":
                return 1,idx
            else:
                return 0,idx
        idx=idx+1
    return 3,idx

### Checks if the string is number or not 
**This function returns true if the string follows the rules given to be a number or else it returns False***

In [5]:
def check_ifNumber(ele):
    ele=ele.lstrip('(')
    ele=ele.rstrip(')')
    if ele=='-' or ele.replace(',','').isdigit() or ele.replace('.','').isdigit():
        return True
    else:
        return False

### Get Absolute Value of the Number
**Based on the rules, this function returns the true value of number <br> 
For eg- (455) --> -455 <br>
        (5.6) --> -5.6 <br>
        (76)  --> 76**

In [6]:
import locale
def get_absolute_number(num):
    if num=="-":
        return num
    if '{' in num or  '}' in num:
        return num
    if "." in num:
        num.replace(",","")
        if num.find("(")!=-1 or num.find(")")!=-1:
            num=num.lstrip("(")
            num=num.rstrip(")")
            #idex=num.find('.')
            #num=num[:idex]
            return "-"+num
        #idex=num.find('.')
        #num=num[:idex]
        return num
    
    flag=0
    if num.find("(")!=-1 or num.find(")")!=-1:
        num=num.lstrip("(")
        num=num.rstrip(")")
        flag=1
    
    locale.setlocale( locale.LC_ALL, 'en_US.UTF-8' ) 
    if flag==1:
        k= locale.atoi(num)
        return -k
    else:
        return locale.atoi(num)

### Try it Yourself !

In [7]:
print(get_absolute_number('(67,887)'))

-67887


### Get Dictionary From Filtered List

In [8]:
import spacy 
nlp = spacy.load('en_core_web_lg')

#if any error regaring spaces....use x=i.split()
def return_dict (cleaned_list,idx,list_of_tags):
    dict1={}
    if idx==1:
        for i in cleaned_list:
            if "STATEMENTS" in i or "Companies Act" in i:
                break
            #x=i.split()
            nlp = spacy.load("en_core_web_sm")
            nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
            doc = nlp(i)
            x=[t.text for t in doc]
            if len(x)>1 and check_ifNumber(x[-2]) and check_ifNumber(x[-1]):
                    orig_num=get_absolute_number(x[-2])
                    key_dict = " ".join(x[:-2])
                    if key_dict=="":
                        key_dict="nan"
                    dict1[key_dict]=str(orig_num)
                    if key_dict!="nan" and key_dict not in list_of_tags:
                        list_of_tags.append(key_dict)
                    #cleaned_list.remove(i)
            else:
                for j in list_of_tags:
                    search_doc=nlp(str(i.lower()))
                    main_doc=nlp(str(j.lower()))
                    if main_doc.similarity(search_doc)>0.85:
                        key=i
                        dict1[key]="nan"
                
        return dict1
    
    elif idx==2:
        for i in cleaned_list:
            if "STATEMENTS" in i or "Companies Act" in i:
                break
            #x=i.split()
            nlp = spacy.load("en_core_web_sm")
            nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
            doc = nlp(i)
            x=[t.text for t in doc]
            if len(x)>1 and check_ifNumber(x[-2]) and check_ifNumber(x[-1]):
                    orig_num=get_absolute_number(x[-1])
                    key_dict = " ".join(x[:-2])
                    if key_dict=="":
                        key_dict="nan"
                    dict1[key_dict]=str(orig_num)
                    if key_dict!="nan" and key_dict not in list_of_tags:
                        list_of_tags.append(key_dict)
                   # cleaned_list.remove(i)
            else:
                for j in list_of_tags:
                    search_doc=nlp(str(i.lower()))
                    main_doc=nlp(str(j.lower()))
                    if main_doc.similarity(search_doc)>0.85:
                        key=i
                        dict1[key]="nan"
        return dict1
    
    elif idx==0:
        for i in cleaned_list:
            if "STATEMENTS" in i or "Companies Act" in i:
                break
            #x=i.split()
            nlp = spacy.load("en_core_web_sm")
            nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
            doc = nlp(i)
            x=[t.text for t in doc]
            if len(x)>1 and check_ifNumber(x[-2]) and check_ifNumber(x[-1]):
                    key_dict = " ".join(x[:-2])
                    if key_dict=="":
                        key_dict="nan"
                    dict1[key_dict]="nan"
                    if key_dict!="nan" and key_dict not in list_of_tags:
                        list_of_tags.append(key_dict)
                    #cleaned_list.remove(i)
            else:
                for j in list_of_tags:
                    search_doc=nlp(str(i.lower()))
                    main_doc=nlp(str(j.lower()))
                    if main_doc.similarity(search_doc)>0.85:
                        key=i
                        dict1[key]="nan"
        return dict1
    
    elif idx==3:
        c=0
        for i in cleaned_list:
            if c==0:
                c=1
                continue
            if "STATEMENTS" in i or "Companies Act" in i:
                break
            #x=i.split()
            nlp = spacy.load("en_core_web_sm")
            nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
            doc = nlp(i)
            x=[t.text for t in doc]
            if len(x)>1 and check_ifNumber(x[-1]):
                    orig_num=get_absolute_number(x[-1])
                    key_dict = " ".join(x[:-1])
                    if key_dict=="":
                        key_dict="nan"
                    dict1[key_dict]=str(orig_num)
                    if key_dict!="nan" and key_dict not in list_of_tags:
                        list_of_tags.append(key_dict)
                    #cleaned_list.remove(i)
            else:
                for j in list_of_tags:
                    search_doc=nlp(str(i.lower()))
                    main_doc=nlp(str(j.lower()))
                    if main_doc.similarity(search_doc)>0.85:
                        key=i
                        dict1[key]="nan"
        
        return dict1
    
    else:
        c=0
        for i in cleaned_list:
            if c==0:
                c=1
                continue
            if "STATEMENTS" in i or "Companies Act" in i:
                break
            #x=i.split()
            nlp = spacy.load("en_core_web_sm")
            nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)
            doc = nlp(i)
            x=[t.text for t in doc]
            if len(x)>1 and check_ifNumber(x[-1]):
                    key_dict = " ".join(x[:-1])
                    if key_dict=="":
                        key_dict="nan"
                    dict1[key_dict]="nan"
                    if key_dict!="nan" and key_dict not in list_of_tags:
                        list_of_tags.append(key_dict)
                #cleaned_list.remove(i)
            else:
                for j in list_of_tags:
                    search_doc=nlp(str(i.lower()))
                    main_doc=nlp(str(j.lower()))
                    if main_doc.similarity(search_doc)>0.85:
                        key=i
                        dict1[key]="nan"
        
        return dict1
        
    

In [9]:
#return list_of_keywords()

### Currency To Html Codes:
**The currency symbol is converted to HTML Codes as given in guidelines**

In [10]:
def return_currency_toHtml(dict1):
    for key in dict1:
        if "$" in key or "€" in key or "£" in key:
            new_key1 = key.replace("$","&#36")
            new_key2=new_key1.replace("€","&#8364")
            new_key3=new_key2.replace("£","&#163")
            val=dict1[key]
            del dict1[key]
            dict1[new_key3]=val
    return dict1

### Write csv
**Function that writes the result to csv file.**

In [11]:
import csv
def write_csv_tofile(row_list):
    with open('hcl_ml_op.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(row_list)

## MAIN CODE

In [12]:
import json
import csv 
import os
from subprocess import call

def main_code():
    row_list = []
    t=[]
    t.append('Filename')
    t.append('Extracted Values')
    row_list.append(t)
    #Use either empty list_of_tags or fill it with whatever tags are know. 
    #Even if it stays empty, it gets filled with all the tags in 'return dict' function.
    list_of_tags=['Issued share capital','Net assets','Current Assets','Fixed Assets',
               'Cash at bank and in hand','Total Shareholder funds','Total assets less current liabilities'
               'Called up share capital not paid','Prepayments and accrued income',
               'Creditors: amounts falling due within one year','Net current assets (liabilities)',
               'Accruals and deferred income','Reserves','Tssued share capital','Tangible assets',
                  'Fully paid called up share capital'
              ]
              
    #list_of_tags=[]
    #c=0
    with open('/home/rk/Downloads/HCL ML Challenge/Results.csv') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            fname=row['Filename']
            filename=fname+'.txt'
            print(filename)
            file_content = read_file(filename)
            res = [clean_lis(i) for i in file_content]
            res = list(filter(('').__ne__, res))
            #print(res)
            idx_2019,idx = check_2019(res)
            print(idx_2019)
            if len(res)>idx-1:
                 del res[:idx+1]
            dict1 = return_dict(res,idx_2019,list_of_tags)
            dict_final=return_currency_toHtml(dict1)
            result = json.dumps(dict_final)
            data = json.loads(result)
            print(data)
            #c=c+1
            temp_list = []
            temp_list.append(str(fname))
            temp_list.append(str(data))
            row_list.append(temp_list)
        write_csv_tofile(row_list)
        #subprocess.call(["sed", "s/'/\"/g", "hcl_ml_op.csv" , "nnn.csv"])
    #print(c)
    #print(set(list_of_tags))
    

## Function call to MAIN CODE

In [13]:
main_code()
#After getting the result in csv file.
#Use:    sed "s/'/\"\"/g"  hcl_ml_op.csv  >  hcl_ml.csv  
#to get final file in correct format 

X8XUG1JD.txt
1
{'Current Assets': '312', 'Creditors: amounts falling due within one year': '-59', 'Net current assets (liabilities)': '253', 'Total assets less current liabilities': '253', 'Total net assets (liabilities)': '253', 'Capital and reserves': '253'}
X8XX0C57.txt
2
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8XX02AX.txt
1
{'Net assets': '0', 'Issued share capital': 'nan', 'Total Shareholder funds': '0', '1 Ordinary Share 0 of &#163 each': '0'}
X8XX0ANN.txt
2
{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8XX0BU1.txt
1
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', 'Total Shareholder funds': '100', '100 Ordinary Shares 1 of &#163 each': '100'}
X8XX0CEA.txt
3
{'Current Assets'

{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', 'Total Shareholder funds': '100', '100 Ordinary Shares of &#163 | each': '100'}
X8XZICNE.txt
2
{'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', 'Total Shareholder funds': '100', '100 Ordinary Shares 1 of &#163 each': '100'}
X8XZICQX.txt
1
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8XZIDG2.txt
3
{'Called up share capital not paid': '-', 'Fixed Assets': '-', 'Current Assets': '-', 'Net current assets (liabilities)': '0', 'Total assets less current liabilities': '0', 'Total net assets (liabilities)': '0', 'Capital and reserves': '0'}
X8XZIDMH.txt
1
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder fun

{'Current Assets': '22560', 'Creditors: amounts falling due within one year': '-2038', 'Net current assets (liabilities)': '20522', 'Total assets less current liabilities': '20522', 'Total net assets (liabilities)': '20522', 'Capital and reserves': '20522'}
X8XZIQEG.txt
1
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8XZIR1E.txt
1
{'Fixed Assets': '46754', 'Current Assets': '8017', 'Creditors: amounts falling due within one year': '-65151', 'Net current assets (liabilities)': '-57134', 'Total assets less current liabilities': '-10380', 'Provisions for liabilities': '-7458', 'Accruals and deferred income': '-1200', 'Total net assets (liabilities)': '-19038', 'Capital and reserves': '-19038'}
X8XZIR3U.txt
3
{'Fixed Assets': '8326', 'Current Assets': '17380', 'Creditors: amounts falling due within one year': '-23723', 'Net current assets (liabilities)': 

{'Current assets': 'nan', 'one year': 'nan', 'Net current assets': 'nan', 'Total assets less current liabilities': 'nan', 'Accruals and deferred income': 'nan', 'Net assets': 'nan', 'Capital and reserves': 'nan', 'Average number of employee': 'nan'}
X8Y2F0HE.txt
3
{'Current assets': '198', 'one year': '3893', 'Net current assets': '4091', 'Total assets less current liabilities': '4091', 'Accruals and deferred income': '-1600', 'Net assets': '2491', 'Capital and reserves': '2491'}
X8Y2F0L6.txt
3
{'Called up share capital not paid': '0', 'Current assets': 'nan', 'Cash at bank and in hand': '100', 'Net assets': '100', 'Issued share capital': 'nan', 'Total shareholder funds': '100', '100 ORDINARY SHARES of &#1631 each': '100'}
X8Y2F3JL.txt
3
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8Y2F3V7.txt
3
{'Called up share capital not paid': '100', 'Net assets

{'Fixed Assets': '32652', 'Current Assets': '13738', 'Prepayments and accrued income': '356', 'Creditors: amounts falling due within one year': '-50429', 'Net current assets (liabilities)': '-36335', 'Total assets less current liabilities': '-3683', 'Total net assets (liabilities)': '-3683', 'Capital and reserves': '-3683'}
X8Y4QWEH.txt
3
{'Current assets': 'nan', 'Cash at bank and in hand': '10', 'Net assets': '10', 'Issued share capital': 'nan', 'Total Shareholder funds': '10', '10 Ordinary Shares 1 of &#163 each': '10'}
X8Y4QX4H.txt
4
{'Current assets': 'nan', 'Cash at bank and in hand': 'nan', 'Net assets': 'nan', 'Issued share capital': 'nan', 'Total Shareholder funds': 'nan', '100 Ordinary Shares of &#163 100 each': 'nan'}
X8Y4QXRM.txt
1
{'Called up share capital not paid': '-', 'Fixed Assets': '-', 'Current Assets': '387', 'Creditors: amounts falling due within one': '-1209', 'Net current assets (liabilities)': '-822', 'Total assets less current liabilities': '-822', 'Total net 

...

{'Current assets': 'nan', 'Cash at bank and in hand': '2', 'Net assets': '2', 'Issued share capital': 'nan', 'Total Shareholder funds': '2', '2 Ordinary Shares 1 of &#163 each': '2'}
X8Y4UI8P.txt
2
{'Called up share capital not paid': '80000', 'Net assets': '80000', 'Tssued share capital': 'nan', 'Total Shareholder funds': '80000', '80000 Ordinary Shares 1 of &#163 each': '80000'}
X8Y4UJ1T.txt
1
{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', 'Total Shareholder funds': '2', '2 Ordinary Shares 1 of &#163 each': '2'}
X8Y4UKE8.txt
1
{'Called up share capital not paid': '100', 'Capital and reserves': '100', 'nan': '2019', 'Employee': '-'}
X8Y4UKFK.txt
4
{'Current assets': 'nan', 'Cash at bank and in hand': 'nan', 'Net assets': 'nan', 'Issued share capital': 'nan', 'Total Shareholder funds': 'nan', '1 ORDINARY Share of &#163 | each': 'nan'}
X8Y4UKFS.txt
1
{'Current Assets': '58357', 'Net current assets (liabilities)': '58187', 'Total assets less c

{'Current assets': '8508', 'one year': '-1616', 'Net current assets': '6892', 'Total assets less current liabilities': '6892', 'Net assets': '6892', 'Capital and reserves': '6892'}
X8Y4V4PZ.txt
1
{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', 'Total Shareholder funds': '100', '10000 Ordinary Shares of &#163 0.01 each': '100'}
X8Y4V5QG.txt
1
{'Called up share capital not paid': '2', 'Net assets': '2', 'Tssued share capital': 'nan', 'Total Shareholder funds': '2', '2 Ordinary Shares 1 of &#163 each': '2'}
X8Y4V6A3.txt
3
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8Y4V6N5.txt
3
{'Fixed Assets': '12000', 'Current Assets': '12901', 'Creditors: amounts falling due within one year': '-24646', 'Net current assets (liabilities)': '-11745', 'Total assets less current liabilities': '255', 'Total net assets (liab

{'Fixed Assets': '1438', 'Current Assets': '25471', 'Creditors: amounts falling due within one year': '-17491', 'Net current assets (liabilities)': '7980', 'Total assets less current liabilities': '9418', 'Total net assets (liabilities)': '9418', 'Capital and reserves': '9418'}
X8Y4VSM9.txt
1
{'Net assets': '0', 'Reserves': '0'}
X8Y4VTL4.txt
1
{'Current Assets': '10582', 'Creditors: amounts falling due within one year': '-10579', 'Net current assets (liabilities)': '3', 'Total assets less current liabilities': '3', 'Total net assets (liabilities)': '3', 'Reserves': '3'}
X8Y4VTZ6.txt
1
{'Current assets': 'nan', 'Cash at bank and in hand': '4', 'Net assets': '4', 'Issued share capital': 'nan', 'Total Shareholder funds': '4', '4 ordinary Shares 1 of &#163 each': '4'}
X8Y4W2I3.txt
1
{'Net assets': '0', 'Reserves': '0'}
X8Y4W2YH.txt
3
{'Current assets': 'nan', 'Cash at bank and in hand': '1', 'Net assets': '1', 'Issued share capital': 'nan', 'Total Shareholder funds': '1', '1000 Ordinary Sh

{'Called up share capital not paid': '1', 'Net assets': '1', 'Tssued share capital': 'nan', 'Total Shareholder funds': '1', '1 Ordinary Share 1 of &#163 each': '1'}
X8Y4SWQ1.txt
4
{'Called up share capital not paid': 'nan', 'Net assets': 'nan', 'Tssued share capital': 'nan', 'Total Shareholder funds': 'nan', '1 100 Shares 1 of &#163 each': 'nan'}
X8Y4SWQH.txt
2
{'Current assets': 'nan', 'Cash at bank and in hand': '2', 'Net assets': '2', 'Issued share capital': 'nan', 'Total Shareholder funds': '2', '2 Ordinary Shares 1 of &#163 each': '2'}
X8Y4SXX7.txt
3
{'Current Assets': '8278', 'Prepayments and accrued income': '1', 'Creditors: amounts falling due within one year': '-8212', 'Net current assets (liabilities)': '67', 'Total assets less current liabilities': '67', 'Total net assets (liabilities)': '67', 'Capital and reserves': '67'}
X8Y4SXZU.txt
2
{'Current assets': 'nan', 'Cash at bank and in hand': '2', 'Net assets': '2', 'Issued share capital': 'nan', 'Total Shareholder funds': '2'

{'Called up share capital not paid': '100', 'Net assets': '100', 'Tssued share capital': 'nan', 'Total Shareholder funds': '100', '100 Ordinary Shares of &#163 | each': '100'}
X8Y4TKPN.txt
1
{'Current Assets': '2124', 'Creditors: amounts falling due within one year': '-250', 'Net current assets (liabilities)': '1874', 'Total assets less current liabilities': '1874', 'nan': '-90.628', 'year': 'nan', 'Total net assets (liabilities)': '-88754', 'Capital and reserves': '-88754'}
X8Y4TLIR.txt
1
{'Called up share capital not paid': '1', 'Current Assets': '320', 'Prepayments and accrued income': '135', 'Net current assets (liabilities)': '455', 'Total assets less current liabilities': '456', 'Total net assets (liabilities)': '456', 'Capital and reserves': '456'}
X8Y4TOYI.txt
1
{'Fixed Assets': '53', 'Current Assets': '3405', 'Creditors: amounts falling due within one year': '-360', 'Net current assets (liabilities)': '3045', 'Total assets less current liabilities': '3098', 'Total net assets (

### After getting the result in csv file.<br> Use:    sed "s/'/\"\"/g"  hcl_ml_op.csv  >  hcl_ml.csv  <br>to get final file in correct format 

**To get result for specific file. Used For Manual Validation** 

In [14]:
list_of_tags=['Issued share capital','Net assets','Current Assets','Fixed Assets',
               'Cash at bank and in hand','Total Shareholder funds','Total assets less current liabilities'
               'Called up share capital not paid','Prepayments and accrued income',
               'Creditors: amounts falling due within one year','Net current assets (liabilities)',
               'Accruals and deferred income','Reserves','Tssued share capital'
              ]
filename='X8XX000W.txt'
file_content = read_file(filename)
res = [clean_lis(i) for i in file_content]
res = list(filter(('').__ne__, res))
print(res)
idx_2019,idx = check_2019(res)
print (idx)
flag=0
if len(res)>idx-1:
    del res[:idx+1]
dict1 = return_dict(res,idx_2019,list_of_tags)
dict_final=return_currency_toHtml(dict1)
#print(dict_final)
result = json.dumps(dict_final)
data = json.loads(result)
print(data)

['Balance Sheet as at 31 October 2019', '2019 2018', '£ £', 'Current assets', 'Cash at bank and in hand 1000 1000', 'Net assets 1000 1000', 'Issued share capital', '1000 Ordinary Shares 1 of £ each 1000 1000', 'Total Shareholder funds 1000 1000', 'STATEMENTS', 'a. For the year ending 31 October 2019 the company was entities to exemption under section 480 of', 'the Companies Act 2006 relating to dormant companies', 'b. The members have not required the company to obtain an audit in accordance with section 476 of', 'the Companies Act 2006.', 'C. The director acknowledge their responsibilities for complying with the requirements of the Act', 'with respect to accounting records and the preparation of accounts.', 'd. These accounts have been prepared in accordance with the provisions applicable to companies', 'subject to the small companies regime.', 'Approved by the Board on | February 2020', 'And signed on their behalf by:', "Liam O'Reilly, Director", 'This document was delivered using el